In [1]:
import peak_detection

In [ ]:
folder_path = "../data/static"
sampling_frequency, num_channels, file_paths = peak_detection.retrieve_parameters(folder_path)
print(sampling_frequency, num_channels, file_paths)

30000.0 128 ['traces_cached_seg0.raw']


In [3]:
channel_locations = peak_detection.retrieve_channel_locations(folder_path + "/probe.json")
print(channel_locations)

[[   0.   20.]
 [   0.   60.]
 [   0.  100.]
 [   0.  140.]
 [   0.  180.]
 [   0.  220.]
 [   0.  260.]
 [   0.  300.]
 [   0.  340.]
 [   0.  380.]
 [   0.  420.]
 [   0.  460.]
 [   0.  500.]
 [   0.  540.]
 [   0.  580.]
 [   0.  620.]
 [   0.  660.]
 [   0.  700.]
 [   0.  740.]
 [   0.  780.]
 [   0.  820.]
 [   0.  860.]
 [   0.  900.]
 [   0.  940.]
 [   0.  980.]
 [   0. 1020.]
 [   0. 1060.]
 [   0. 1100.]
 [   0. 1140.]
 [   0. 1180.]
 [   0. 1220.]
 [   0. 1260.]
 [  16.    0.]
 [  16.   40.]
 [  16.   80.]
 [  16.  120.]
 [  16.  160.]
 [  16.  200.]
 [  16.  240.]
 [  16.  280.]
 [  16.  320.]
 [  16.  360.]
 [  16.  400.]
 [  16.  440.]
 [  16.  480.]
 [  16.  520.]
 [  16.  560.]
 [  16.  600.]
 [  16.  640.]
 [  16.  680.]
 [  16.  720.]
 [  16.  760.]
 [  16.  800.]
 [  16.  840.]
 [  16.  880.]
 [  16.  920.]
 [  16.  960.]
 [  16. 1000.]
 [  16. 1040.]
 [  16. 1080.]
 [  16. 1120.]
 [  16. 1160.]
 [  16. 1200.]
 [  16. 1240.]
 [  32.   20.]
 [  32.   60.]
 [  32.  1

In [4]:
chunk_iterator = []
for seg in file_paths:
    chunk_iterator.append(peak_detection.FileChunkIterator(folder_path + "/" + seg, int(sampling_frequency/10), num_channels))
print(chunk_iterator)

In [5]:
peaks = []
for seg in chunk_iterator:
    for chunk in seg: # Each chunk is a numpy array of shape (chunk_size, num_channels)
        peaks = peaks + peak_detection.rust_peak_detection_locally_exclusive(chunk, channel_locations, sampling_frequency)
        if len(peaks) > 1000:
            break
    print(len(peaks))

1074


In [6]:
# we benchmark rust_peak_detection_locally_exclusive
import timeit
mean_time = timeit.timeit('peak_detection.rust_peak_detection_locally_exclusive(next(chunk_iterator[0]), channel_locations, sampling_frequency)', globals=globals(), number=10)/10
print("Temps moyen d'exécution :", mean_time)

Temps moyen d'exécution : 0.00395267999265343


In [7]:
import cProfile
cProfile.run('peak_detection.rust_peak_detection_locally_exclusive(next(chunk_iterator[0]), channel_locations, sampling_frequency)')

         5 function calls in 0.004 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    0.004    0.004 <string>:1(<module>)
        1    0.000    0.000    0.004    0.004 {built-in method builtins.exec}
        1    0.001    0.001    0.001    0.001 {built-in method builtins.next}
        1    0.003    0.003    0.003    0.003 {built-in method peak_detection.peak_detection.rust_peak_detection_locally_exclusive}
        1    0.000    0.000    0.000    0.000 {method 'disable' of '_lsprof.Profiler' objects}


